In [1]:
import gc
import os
import sys
import math
import random
import warnings
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)
import optuna
from tqdm import tqdm
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from pandarallel import pandarallel
# pandarallel.initialize()
warnings.simplefilter("ignore")
import ctypes as ct
from datetime import timedelta

from sklearn.metrics import mean_absolute_error

import statistics as st
import lightgbm as lgbm
from scipy.stats import norm

In [2]:
sys.path.append('../../')
import src.utils as utils

## Param

In [3]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName', 'birthCity', 'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
transactions_cols = ['playerId', 'transaction_flag', 'date']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date']

feature_cols1 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info'] 

feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info'] 

feature_cols3 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info'] 

feature_cols4 = ['week_day', 'annual_day', 'month', 'label_playerId', 'label_primaryPositionName', 'label_teamId', 'label_birthCity',
                'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',
        "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
         "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt", 
        'season_info'] 

In [4]:
# training lightgbm
params1 = {'objective':'mae',
           'reg_alpha': 0.14947461820098767, 
           'reg_lambda': 0.10185644384043743, 
           'n_estimators': 3633, 
           'learning_rate': 0.08046301304430488, 
           'num_leaves': 674, 
           'feature_fraction': 0.9101240539122566, 
           'bagging_fraction': 0.9884451442950513, 
           'bagging_freq': 8, 
           'min_child_samples': 51}

params2 = {'objective':'mae',
           'reg_alpha': 0.1,
           'reg_lambda': 0.1, 
           'n_estimators': 80,
           'learning_rate': 0.1,
           'random_state': 42,
           "num_leaves": 22}

params3 = {'objective':'mae',
           'reg_alpha': 0.1,
           'reg_lambda': 0.1, 
           'n_estimators': 10000,
           'learning_rate': 0.1,
           'random_state': 42,
           "num_leaves": 100}

params4 = {'objective':'mae',
           'reg_alpha': 0.016468100279441976, 
           'reg_lambda': 0.09128335764019105, 
           'n_estimators': 9868, 
           'learning_rate': 0.10528150510326864, 
           'num_leaves': 157, 
           'feature_fraction': 0.5419185713426886, 
           'bagging_fraction': 0.2637405128936662, 
           'bagging_freq': 19, 
           'min_child_samples': 71}

In [5]:
EXP_NUM = 45
NFOLDS = 5
SEED = 42

In [6]:
utils.set_seed(SEED)

## Dir

In [7]:
DATA_DIR = Path("/home/knikaido/work/MLB-Player-Digital-Engagement-Forecasting/data/")
MAIN_DATA_DIR = DATA_DIR / 'mlb-player-digital-engagement-forecasting'
TRAIN_DIR = MAIN_DATA_DIR / 'train'
OUTPUT_DIR = Path('./output/')

In [8]:
players = pd.read_csv(MAIN_DATA_DIR / 'players.csv')

rosters = pd.read_csv(TRAIN_DIR / 'rosters_train.csv')
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv')
scores = pd.read_csv(TRAIN_DIR / 'playerBoxScores_train.csv')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
seasons = pd.read_csv(MAIN_DATA_DIR / 'seasons.csv')

# events = pd.read_csv(TRAIN_DIR / 'events_train.csv')
# events = events.groupby(['gameDate']).sum().reset_index()


In [9]:
scores

,playerId,date,home,gamePk,teamId,jerseyNum,positionCode,battingOrder,gamesPlayedBatting,flyOuts,...,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,index
0,112526,20180402,0,529469,140,40.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,91
1,112526,20180408,1,529546,140,40.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,97
2,112526,20180410,1,529565,140,40.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,99
3,112526,20180415,0,529640,140,40.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,104
4,112526,20180421,1,529718,140,40.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182945,685503,20210409,1,634478,140,35.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1194
182946,685503,20210414,0,634496,140,35.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1199
182947,685503,20210419,0,634536,140,35.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1204
182948,685503,20210425,0,634393,140,35.0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1210


In [10]:
seasons = seasons.fillna('0000-00-00')
for c_ in seasons.columns[1:]:
    seasons[c_] = seasons[c_].str.replace('-', '').astype(int)

In [11]:
game_dates_range = []
game_dates_range.append(np.array([20170101, 20171231]))
game_dates_range.append(scores[scores['date'] <= 20181231].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range.append(scores[(scores['date'] >= 20190101) & (scores['date'] <= 20191231)].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range.append(scores[(scores['date'] >= 20200101) & (scores['date'] <= 20201231)].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range.append(scores[(scores['date'] >= 20210101)].sort_values('date')['date'].iloc[[0, -1]].values)
game_dates_range = np.array(game_dates_range)

In [12]:
seasons['gameStartDate'] = game_dates_range[:, 0]
seasons['gameEndDate'] = game_dates_range[:, 1]

In [13]:
seasons

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate,gameStartDate,gameEndDate
0,2017,20170402,20171101,20170222,20170401,20170402,20171001,20170709,20170711,20170714,20171003,20171101,20170101,20171231
1,2018,20180329,20181028,20180221,20180327,20180329,20181001,20180715,20180717,20180719,20181002,20181028,20180329,20181028
2,2019,20190320,20191030,20190221,20190326,20190320,20190929,20190707,20190709,20190711,20191001,20191030,20190320,20191030
3,2020,20200723,20201028,20200221,20200722,20200723,20200927,20200825,0,20200826,20200929,20201028,20200723,20201027
4,2021,20210228,20211031,20210228,20210330,20210401,20211003,20210711,20210713,20210715,20211004,20211031,20210401,20210430


In [14]:
targets_train = targets[(targets['date'] >= 20210401)]

In [15]:
targets_train

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date
2444346,2021-04-02,624415,2.810640,3.998462,0.136137,10.012438,1186,20210401
2444347,2021-04-02,656548,0.000000,0.153787,0.068889,0.062189,1186,20210401
2444348,2021-04-02,400284,0.000971,0.010985,0.021323,3.182007,1186,20210401
2444349,2021-04-02,606157,0.002736,1.642226,0.021323,0.549337,1186,20210401
2444350,2021-04-02,665620,0.020473,0.411929,0.549469,0.435323,1186,20210401
...,...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430


In [16]:
playerId_list = targets_train['playerId'].unique()

In [17]:
def calc_corr(df):
    # 相関係数行列を作成
    corr_mat = df.corr(method='pearson')

    # 行（列）サイズを取得
    n = corr_mat.shape[0]
    corr_ary = []

    for i in range(n):
        for j in range(i):
            if i == j:
                continue
            corr_ary.append(corr_mat.iloc[i,j])

    return corr_ary

In [18]:
def calc_probs(pid,df,temp):
    to_append=[pid,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    targets=['target1','target2','target3','target4']
    z=1
    for target in targets:
        target_prob = temp[target].tolist()
        mean = np.mean(target_prob)
        std = np.std(target_prob)
        median = st.median(target_prob)
        distribution = norm(mean, std)
        min_weight = min(target_prob)
        max_weight = max(target_prob)
        values = list(np.linspace(min_weight, max_weight))
        probabilities = [distribution.pdf(v) for v in values]
        max_value = max(probabilities)
        max_index = probabilities.index(max_value)
        to_append[z]=mean
        to_append[z+1]=median
        to_append[z+2]=std
        to_append[z+3]=min_weight
        to_append[z+4]=max_weight
        to_append[z+5]=target_prob[max_index]
        to_append[z+6]=temp[target].skew()
        to_append[z+7]=temp[target].kurt()

        z=z+8
    corr_ = calc_corr(temp[['target1', 'target2', 'target3', 'target4']])
    to_append[z:] = corr_  
    df_length = len(df)
    df.loc[df_length] = to_append
    return df
    

### CREATE DATAFRAME to store probabilities
column_names = ["playerId", "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_prob","target1_skew","target1_kurt",
                "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_prob","target2_skew","target2_kurt",
                "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_prob","target3_skew","target3_kurt",
                "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_prob","target4_skew","target4_kurt",
                'tgt1_2_corr', 'tgt1_3_corr', 'tgt2_3_corr', 'tgt1_4_corr', 'tgt2_4_corr', 'tgt3_4_corr']
player_target_probs = pd.DataFrame(columns = column_names)
    
for pid in tqdm(playerId_list):
    temp = targets_train[targets_train['playerId'] == pid]
    player_target_stats=calc_probs(pid,player_target_probs,temp)

100%|██████████| 2061/2061 [00:46<00:00, 44.73it/s]


In [19]:
player_target_stats

,playerId,target1_mean,target1_median,target1_std,target1_min,target1_max,target1_prob,target1_skew,target1_kurt,target2_mean,...,target4_max,target4_prob,target4_skew,target4_kurt,tgt1_2_corr,tgt1_3_corr,tgt2_3_corr,tgt1_4_corr,tgt2_4_corr,tgt3_4_corr
0,624415.0,1.193031,0.489252,1.578499,0.008327,7.267988,0.785380,2.208850,6.071897,5.398585,...,26.025186,10.464387,1.695464,3.831404,0.408492,-0.009458,-0.101143,0.227373,0.694767,-0.039737
1,656548.0,0.110672,0.001015,0.428984,0.000000,2.392265,0.000000,5.272732,28.385845,0.445853,...,0.423573,0.032978,1.294196,1.140084,0.566411,0.588373,0.527886,0.231260,0.525065,0.132559
2,400284.0,0.001807,0.001301,0.001164,0.000437,0.004770,0.001127,1.155729,0.582752,0.073071,...,8.909871,7.882136,0.388255,-0.703079,0.102451,0.429495,0.046450,0.006606,0.277479,-0.016683
3,606157.0,0.748247,0.024461,1.318711,0.002736,4.840508,0.548613,2.003263,3.235374,2.027612,...,3.931330,0.231750,2.053818,4.653597,0.641382,0.093253,-0.002014,0.404372,0.626199,0.249109
4,665620.0,0.483331,0.020388,1.415708,0.004932,6.322554,0.128698,3.507992,11.893073,1.913634,...,8.474798,0.314951,4.910653,25.637658,0.807310,0.810125,0.965737,0.778467,0.954381,0.959152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,621350.0,0.000020,0.000000,0.000051,0.000000,0.000175,0.000175,2.331868,3.822563,0.007298,...,0.206009,0.017662,0.992179,0.627281,0.278321,NaN,NaN,-0.041837,0.060094,NaN
2057,665759.0,0.000095,0.000000,0.000174,0.000000,0.000701,0.000000,2.033176,3.899046,0.238824,...,0.332631,0.019864,2.366135,7.230272,0.676003,NaN,NaN,0.678376,0.848313,NaN
2058,448178.0,0.000020,0.000000,0.000052,0.000000,0.000174,0.000000,2.345681,3.902834,0.005605,...,0.350299,0.188265,2.416382,8.149779,-0.053275,-0.134373,-0.260662,-0.106479,-0.172380,-0.078929
2059,488681.0,0.000074,0.000000,0.000142,0.000000,0.000506,0.000000,2.153297,3.865929,0.012644,...,0.453328,0.075379,1.055577,1.166366,-0.114731,0.364257,0.095866,0.216720,0.301329,0.054098


In [20]:
data_names=player_target_stats.columns.values.tolist()

In [21]:
players['DOY'] = pd.to_datetime(players['DOB'], format="%Y-%m-%d").dt.year
players['mlbDebutYear'] = pd.to_datetime(players['mlbDebutDate'], format="%Y-%m-%d").dt.year
players['DebutAge'] = players['mlbDebutYear'] - players['DOY']

In [22]:
# creat dataset
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])


In [23]:
# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
birthCityn2num = {c: i for i, c in enumerate(train['birthCity'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_birthCity'] = train['birthCity'].map(birthCityn2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)

In [24]:
date_ = pd.to_datetime(train['date'], format="%Y%m%d")
train['annual_day'] = (date_ - pd.to_datetime(date_.dt.year, format="%Y")) /  timedelta(days=1)
train['week_day'] = date_.dt.weekday
train['month'] = date_.dt.month

In [25]:
train['gameday'] = ~train['battingOrder'].isna()*1

In [26]:
train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

In [27]:
def count_consecutive_items_n_cols(df, col_name_list, output_col):
    cum_sum_list = [
        (df[col_name] != df[col_name].shift(1)).cumsum().tolist() for col_name in col_name_list
    ]
    df[output_col] = df.groupby(
        ["_".join(map(str, x)) for x in zip(*cum_sum_list)]
    ).cumcount() + 1
    return df

In [28]:
train=count_consecutive_items_n_cols(train,['playerId','gameday'],'daysSinceLastGame')
train.loc[train['gameday']==1,'daysSinceLastGame']=0

In [29]:
train_game = train[train['gameday']==1]
train_last_game = train_game[~train_game.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
train_last_game.columns = ['playerId', 'lastdate']
train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
train_last_game = pd.merge(train_player_unique, train_last_game, on=['playerId'], how='left' )
train_last_game = train_last_game.fillna(20171231)

In [30]:
train_last_game.sort_values('lastdate')

,playerId,lastdate
856,593590,20171231.0
1995,670462,20171231.0
2000,670764,20171231.0
1765,661269,20171231.0
1932,667674,20171231.0
...,...,...
473,542932,20210430.0
1480,641856,20210430.0
1481,641857,20210430.0
483,543037,20210430.0


In [31]:
def extract_season(date_raw, season_start_end):
    idxes = 0
    for raw in season_start_end.iloc():
        idx_ = ((date_raw >= raw.iloc[0]) & (date_raw <= raw.iloc[1])) * 1
        idxes += idx_
    return idxes

In [32]:
on_preseason_idxes = extract_season(train['date'], seasons[['preSeasonStartDate', 'preSeasonEndDate']])
on_season_idxes = extract_season(train['date'], seasons[['regularSeasonStartDate', 'regularSeasonEndDate']]) * 2
on_postseason_idxes = extract_season(train['date'], seasons[['postSeasonStartDate', 'postSeasonEndDate']]) * 3

special_days = seasons['lastDate1stHalf'].to_list() + seasons['allStarDate'].to_list() + seasons['firstDate2ndHalf'].to_list()
special_idxes = 0
for day in special_days:
    special_idxes += (train['date'] == day) * 4

on_total_season_idxes = on_preseason_idxes
on_total_season_idxes[on_season_idxes==2] = 2
on_total_season_idxes[on_postseason_idxes==3] = 3
on_total_season_idxes[special_idxes==4] = 4

train['season_info'] = on_total_season_idxes

In [33]:
## only on season
on_whole_idxes = extract_season(train['date'], seasons[['seasonStartDate', 'seasonEndDate']])
train = train[on_whole_idxes == 1].reset_index(drop=True)

In [34]:
# train = train[train['playerForTestSetAndFuturePreds']==True].reset_index(drop=True)

In [35]:
train

,playerId,target1,target2,target3,target4,date,primaryPositionName,birthCity,DOY,mlbDebutYear,...,label_primaryPositionName,label_birthCity,label_teamId,label_status,annual_day,week_day,month,gameday,daysSinceLastGame,season_info
0,112526,0.031761,2.731418,0.388556,6.349412,20180329,Pitcher,Altamira,1973,1997.0,...,0,588,1,3,87.0,3,3,0,88,2
1,112526,0.025906,4.622162,0.408017,11.508375,20180330,Pitcher,Altamira,1973,1997.0,...,0,588,1,3,88.0,4,3,0,89,2
2,112526,0.053185,4.767842,0.275408,14.600851,20180331,Pitcher,Altamira,1973,1997.0,...,0,588,1,3,89.0,5,3,0,90,2
3,112526,0.771100,63.601677,7.566316,100.000000,20180401,Pitcher,Altamira,1973,1997.0,...,0,588,1,3,90.0,6,4,0,91,2
4,112526,5.957846,22.427930,33.900803,38.857939,20180402,Pitcher,Altamira,1973,1997.0,...,0,588,7,0,91.0,0,4,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234534,685503,0.044617,1.224728,0.009437,0.737463,20210426,Pitcher,Hiroshima,1992,2021.0,...,0,902,7,0,115.0,0,4,0,1,2
1234535,685503,0.019123,1.178880,0.013161,0.790301,20210427,Pitcher,Hiroshima,1992,2021.0,...,0,902,7,0,116.0,1,4,0,2,2
1234536,685503,0.015799,4.323489,0.002350,0.970273,20210428,Pitcher,Hiroshima,1992,2021.0,...,0,902,7,0,117.0,2,4,0,3,2
1234537,685503,0.018770,31.946021,0.305491,5.938273,20210429,Pitcher,Hiroshima,1992,2021.0,...,0,902,7,0,118.0,3,4,0,4,2


In [36]:
# def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
#     oof_pred = np.zeros(len(y_valid), dtype=np.float32)
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(X_train, y_train), (x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
#     oof_pred = model.predict(x_valid)
#     oof_pred = np.clip(oof_pred, 0, 100)
#     score = mean_absolute_error(oof_pred, y_valid)
#     print('mae:', score)
#     return oof_pred, model, score


In [37]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, learning_rates, verbose=100):
    def opt(trial):
        params = {
                'random_state': SEED,
                'objective':'mae',
                'n_estimators': 10000,
                'learning_rate': 0.1,
                'max_depth': trial.suggest_int('max_depth', 1, 20),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
                'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
                'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
                'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
        }

        model_opt = lgbm.LGBMRegressor(**params)

        model_opt.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_valid, y_valid)],  
            early_stopping_rounds=verbose, 
            verbose=verbose)
        oof_pred = model_opt.predict(x_valid)
        oof_pred = np.clip(oof_pred, 0, 100)
        score = mean_absolute_error(oof_pred, y_valid)
        return -score
    return opt

In [38]:
train_X = train
train_y = train[['target1', 'target2', 'target3', 'target4']]
tr_idxs = []
val_idxs = []

In [39]:
# tr_idx = (train['date'].astype(int) < 20200801)
# val_idx = (train['date'].astype(int) >= 20200801) & (train['date'].astype(int) < 20200901)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

# tr_idx = (train['date'].astype(int) < 20200901)
# val_idx = (train['date'].astype(int) >= 20200901) & (train['date'].astype(int) < 20201001)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

# tr_idx = (train['date'].astype(int) < 20201001)
# val_idx = (train['date'].astype(int) >= 20201001) & (train['date'].astype(int) < 20201028)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

# tr_idx = (train['date'].astype(int) < 20210228)
# val_idx = (train['date'].astype(int) >= 20210228) & (train['date'].astype(int) < 20210401)
# tr_idxs.append(tr_idx)
# val_idxs.append(val_idx)

tr_idx = (train['date'].astype(int) < 20210401)
val_idx = ~tr_idx
tr_idxs.append(tr_idx)
val_idxs.append(val_idx)

In [40]:
idx = 0

tr_idx = tr_idxs[idx]
val_idx = val_idxs[idx]

x_train = train_X.loc[tr_idx].reset_index(drop=True)
y_train = train_y.loc[tr_idx].reset_index(drop=True)
x_valid = train_X.loc[val_idx].reset_index(drop=True)
y_valid = train_y.loc[val_idx].reset_index(drop=True)

In [41]:
learning_rates = [0.1, 0.1, 0.1, 0.1]

In [42]:
study1 = optuna.create_study(direction='maximize')
study1.optimize(fit_lgbm(x_train[feature_cols1], y_train['target1'], x_valid[feature_cols1], y_valid['target1'], learning_rates[0]), n_trials=100)

print('Number of finished trials:', len(study1.trials))
print('Best trial:', study1.best_trial.params)

[I 2021-07-06 13:27:06,920] A new study created in memory with name: no-name-33ad756c-4dc8-4bd4-9124-af915d5df8db


[LightGBM] [Warning] feature_fraction is set=0.597160489124598, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.597160489124598
[LightGBM] [Warning] bagging_fraction is set=0.7172530950966584, subsample=0.6 will be ignored. Current value: bagging_fraction=0.7172530950966584
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.627695	valid_1's l1: 0.7147
[200]	training's l1: 0.627689	valid_1's l1: 0.714689


[I 2021-07-06 13:27:18,426] Trial 0 finished with value: -0.7146888932629226 and parameters: {'max_depth': 15, 'min_child_weight': 11, 'subsample': 0.6, 'colsample_bytree': 0.7, 'reg_lambda': 1.6172414935067496, 'reg_alpha': 46.41237033844323, 'feature_fraction': 0.597160489124598, 'bagging_fraction': 0.7172530950966584, 'bagging_freq': 3}. Best is trial 0 with value: -0.7146888932629226.


Early stopping, best iteration is:
[137]	training's l1: 0.627689	valid_1's l1: 0.714689
[LightGBM] [Warning] feature_fraction is set=0.6587024169385305, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.6587024169385305
[LightGBM] [Warning] bagging_fraction is set=0.6947336823126458, subsample=0.8 will be ignored. Current value: bagging_fraction=0.6947336823126458
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.630349	valid_1's l1: 0.716498
[200]	training's l1: 0.629479	valid_1's l1: 0.714389
[300]	training's l1: 0.629459	valid_1's l1: 0.71435
[400]	training's l1: 0.629455	valid_1's l1: 0.714347


[I 2021-07-06 13:27:35,190] Trial 1 finished with value: -0.714344020362798 and parameters: {'max_depth': 15, 'min_child_weight': 3, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_lambda': 58.08419221987469, 'reg_alpha': 231.46316440893403, 'feature_fraction': 0.6587024169385305, 'bagging_fraction': 0.6947336823126458, 'bagging_freq': 7}. Best is trial 1 with value: -0.714344020362798.


...

Early stopping, best iteration is:
[542]	training's l1: 0.622357	valid_1's l1: 0.703836
[LightGBM] [Warning] feature_fraction is set=0.9252838420153824, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9252838420153824
[LightGBM] [Warning] bagging_fraction is set=0.4077372953699203, subsample=0.7 will be ignored. Current value: bagging_fraction=0.4077372953699203
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.617426	valid_1's l1: 0.694741
[200]	training's l1: 0.61606	valid_1's l1: 0.692772
[300]	training's l1: 0.615866	valid_1's l1: 0.692574


[I 2021-07-06 14:07:49,681] Trial 98 finished with value: -0.6925698907647723 and parameters: {'max_depth': 5, 'min_child_weight': 10, 'subsample': 0.7, 'colsample_bytree': 0.8, 'reg_lambda': 0.01128812171247567, 'reg_alpha': 0.5026783718376314, 'feature_fraction': 0.9252838420153824, 'bagging_fraction': 0.4077372953699203, 'bagging_freq': 6}. Best is trial 80 with value: -0.6744820355645097.


Early stopping, best iteration is:
[272]	training's l1: 0.615881	valid_1's l1: 0.692573
[LightGBM] [Warning] feature_fraction is set=0.8931427821485273, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.8931427821485273
[LightGBM] [Warning] bagging_fraction is set=0.4561639285855082, subsample=0.5 will be ignored. Current value: bagging_fraction=0.4561639285855082
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.642587	valid_1's l1: 0.737402
[200]	training's l1: 0.633382	valid_1's l1: 0.721607
[300]	training's l1: 0.632427	valid_1's l1: 0.720406
[400]	training's l1: 0.632302	valid_1's l1: 0.720315
[500]	training's l1: 0.63178	valid_1's l1: 0.719267
[600]	training's l1: 0.631759	valid_1's l1: 0.719231
[700]	training's l1: 0.631379	valid_1's l1: 0.718645
[800]	training's l1: 0.630357	valid_1's l1: 0.716744
[900]	training's l

[I 2021-07-06 14:08:19,420] Trial 99 finished with value: -0.7131150826446087 and parameters: {'max_depth': 3, 'min_child_weight': 13, 'subsample': 0.5, 'colsample_bytree': 0.7, 'reg_lambda': 6.6238146055988505, 'reg_alpha': 1.099113728491789, 'feature_fraction': 0.8931427821485273, 'bagging_fraction': 0.4561639285855082, 'bagging_freq': 4}. Best is trial 80 with value: -0.6744820355645097.


Number of finished trials: 100
Best trial: {'max_depth': 5, 'min_child_weight': 15, 'subsample': 0.6, 'colsample_bytree': 0.8, 'reg_lambda': 0.4069187916380186, 'reg_alpha': 0.3230034165640419, 'feature_fraction': 0.930062038668566, 'bagging_fraction': 0.46153099230093236, 'bagging_freq': 3}


In [43]:
study2 = optuna.create_study(direction='maximize')
study2.optimize(fit_lgbm(x_train[feature_cols2], y_train['target2'], x_valid[feature_cols2], y_valid['target2'], learning_rates[1]), n_trials=100)

print('Number of finished trials:', len(study2.trials))
print('Best trial:', study2.best_trial.params)

[I 2021-07-06 14:08:19,446] A new study created in memory with name: no-name-41a95a91-73c4-46a4-afcb-fb631dc5b505


[LightGBM] [Warning] feature_fraction is set=0.6301726147901224, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.6301726147901224
[LightGBM] [Warning] bagging_fraction is set=0.8540497693650837, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8540497693650837
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.46939	valid_1's l1: 1.36788


[I 2021-07-06 14:08:27,866] Trial 0 finished with value: -1.3635721001649752 and parameters: {'max_depth': 5, 'min_child_weight': 14, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_lambda': 0.0412905071864811, 'reg_alpha': 0.0062128386456524005, 'feature_fraction': 0.6301726147901224, 'bagging_fraction': 0.8540497693650837, 'bagging_freq': 2}. Best is trial 0 with value: -1.3635721001649752.


Early stopping, best iteration is:
[32]	training's l1: 1.5222	valid_1's l1: 1.36357
[LightGBM] [Warning] feature_fraction is set=0.9471820685363787, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9471820685363787
[LightGBM] [Warning] bagging_fraction is set=0.9727183110845958, subsample=0.8 will be ignored. Current value: bagging_fraction=0.9727183110845958
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.46941	valid_1's l1: 1.383


[I 2021-07-06 14:08:36,169] Trial 1 finished with value: -1.370911844096845 and parameters: {'max_depth': 11, 'min_child_weight': 11, 'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_lambda': 0.011578209880924163, 'reg_alpha': 0.014143536252848731, 'feature_fraction': 0.9471820685363787, 'bagging_fraction': 0.9727183110845958, 'bagging_freq': 4}. Best is trial 0 with value: -1.3635721001649752.


...

Early stopping, best iteration is:
[86]	training's l1: 1.48863	valid_1's l1: 1.36028
[LightGBM] [Warning] feature_fraction is set=0.707162797970143, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.707162797970143
[LightGBM] [Warning] bagging_fraction is set=0.9600721498605378, subsample=0.5 will be ignored. Current value: bagging_fraction=0.9600721498605378
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.4638	valid_1's l1: 1.3795


[I 2021-07-06 14:25:30,110] Trial 98 finished with value: -1.3720069241854105 and parameters: {'max_depth': 6, 'min_child_weight': 17, 'subsample': 0.5, 'colsample_bytree': 0.6, 'reg_lambda': 0.017101718245946248, 'reg_alpha': 2.0615647255099003, 'feature_fraction': 0.707162797970143, 'bagging_fraction': 0.9600721498605378, 'bagging_freq': 10}. Best is trial 55 with value: -1.3567368068404055.


Early stopping, best iteration is:
[35]	training's l1: 1.50474	valid_1's l1: 1.37201
[LightGBM] [Warning] feature_fraction is set=0.8362627235356972, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.8362627235356972
[LightGBM] [Warning] bagging_fraction is set=0.9109512127186652, subsample=0.5 will be ignored. Current value: bagging_fraction=0.9109512127186652
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.46786	valid_1's l1: 1.36714


[I 2021-07-06 14:25:36,697] Trial 99 finished with value: -1.3595646298403141 and parameters: {'max_depth': 5, 'min_child_weight': 16, 'subsample': 0.5, 'colsample_bytree': 0.6, 'reg_lambda': 0.00428632733857397, 'reg_alpha': 1.0929869084705977, 'feature_fraction': 0.8362627235356972, 'bagging_fraction': 0.9109512127186652, 'bagging_freq': 9}. Best is trial 55 with value: -1.3567368068404055.


Early stopping, best iteration is:
[39]	training's l1: 1.49716	valid_1's l1: 1.35956
Number of finished trials: 100
Best trial: {'max_depth': 5, 'min_child_weight': 18, 'subsample': 0.5, 'colsample_bytree': 0.6, 'reg_lambda': 0.003047336161981325, 'reg_alpha': 0.06817362969639486, 'feature_fraction': 0.9298564165879744, 'bagging_fraction': 0.9439577588582982, 'bagging_freq': 9}


In [44]:
study3 = optuna.create_study(direction='maximize')
study3.optimize(fit_lgbm(x_train[feature_cols3], y_train['target3'], x_valid[feature_cols3], y_valid['target3'], learning_rates[2]), n_trials=100)

print('Number of finished trials:', len(study3.trials))
print('Best trial:', study3.best_trial.params)

[I 2021-07-06 14:25:36,719] A new study created in memory with name: no-name-cc7b3a60-1f26-4a95-8b99-0f5eb67fa4ce


[LightGBM] [Warning] feature_fraction is set=0.5963391368070248, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.5963391368070248
[LightGBM] [Warning] bagging_fraction is set=0.8703301390851299, subsample=0.6 will be ignored. Current value: bagging_fraction=0.8703301390851299
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.753462	valid_1's l1: 0.552287
[200]	training's l1: 0.753181	valid_1's l1: 0.552019
[300]	training's l1: 0.752613	valid_1's l1: 0.551487
[400]	training's l1: 0.752612	valid_1's l1: 0.551485
[500]	training's l1: 0.752356	valid_1's l1: 0.551286
[600]	training's l1: 0.752341	valid_1's l1: 0.551237
Early stopping, best iteration is:
[559]	training's l1: 0.752343	valid_1's l1: 0.551233


[I 2021-07-06 14:26:07,374] Trial 0 finished with value: -0.5512322544000922 and parameters: {'max_depth': 18, 'min_child_weight': 1, 'subsample': 0.6, 'colsample_bytree': 0.7, 'reg_lambda': 7.8186975570770985, 'reg_alpha': 0.4618322073505791, 'feature_fraction': 0.5963391368070248, 'bagging_fraction': 0.8703301390851299, 'bagging_freq': 7}. Best is trial 0 with value: -0.5512322544000922.


[LightGBM] [Warning] feature_fraction is set=0.696360352955839, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.696360352955839
[LightGBM] [Warning] bagging_fraction is set=0.6680421308074513, subsample=0.5 will be ignored. Current value: bagging_fraction=0.6680421308074513
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.754167	valid_1's l1: 0.552326
[200]	training's l1: 0.752882	valid_1's l1: 0.551123
[300]	training's l1: 0.752081	valid_1's l1: 0.550334
[400]	training's l1: 0.751711	valid_1's l1: 0.550236


[I 2021-07-06 14:26:24,370] Trial 1 finished with value: -0.5502244617509805 and parameters: {'max_depth': 15, 'min_child_weight': 2, 'subsample': 0.5, 'colsample_bytree': 0.9, 'reg_lambda': 0.34107126967003454, 'reg_alpha': 0.16887150026165715, 'feature_fraction': 0.696360352955839, 'bagging_fraction': 0.6680421308074513, 'bagging_freq': 4}. Best is trial 1 with value: -0.5502244617509805.


...

Early stopping, best iteration is:
[537]	training's l1: 0.748915	valid_1's l1: 0.547328
[LightGBM] [Warning] feature_fraction is set=0.5125599832347246, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.5125599832347246
[LightGBM] [Warning] bagging_fraction is set=0.6946653167740754, subsample=0.9 will be ignored. Current value: bagging_fraction=0.6946653167740754
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.753125	valid_1's l1: 0.552274
[200]	training's l1: 0.752647	valid_1's l1: 0.552172
[300]	training's l1: 0.752291	valid_1's l1: 0.551697


[I 2021-07-06 14:52:35,896] Trial 98 finished with value: -0.5516883305036164 and parameters: {'max_depth': 14, 'min_child_weight': 9, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 0.0027606483244542275, 'reg_alpha': 9.686171123609908, 'feature_fraction': 0.5125599832347246, 'bagging_fraction': 0.6946653167740754, 'bagging_freq': 9}. Best is trial 87 with value: -0.5443812101222176.


Early stopping, best iteration is:
[241]	training's l1: 0.752305	valid_1's l1: 0.551689
[LightGBM] [Warning] feature_fraction is set=0.41283925439950736, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.41283925439950736
[LightGBM] [Warning] bagging_fraction is set=0.7132442293934276, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7132442293934276
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.748949	valid_1's l1: 0.548443
[200]	training's l1: 0.747868	valid_1's l1: 0.547333
[300]	training's l1: 0.74748	valid_1's l1: 0.547006
[400]	training's l1: 0.747394	valid_1's l1: 0.546977
[500]	training's l1: 0.746997	valid_1's l1: 0.546675
[600]	training's l1: 0.746953	valid_1's l1: 0.54666


[I 2021-07-06 14:52:59,385] Trial 99 finished with value: -0.5466447093647719 and parameters: {'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 0.012810218114441926, 'reg_alpha': 1.5295133612669514, 'feature_fraction': 0.41283925439950736, 'bagging_fraction': 0.7132442293934276, 'bagging_freq': 4}. Best is trial 87 with value: -0.5443812101222176.


Early stopping, best iteration is:
[547]	training's l1: 0.746959	valid_1's l1: 0.546652
Number of finished trials: 100
Best trial: {'max_depth': 5, 'min_child_weight': 8, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 0.018839288270763987, 'reg_alpha': 1.760156672313524, 'feature_fraction': 0.4601637468609972, 'bagging_fraction': 0.6761793533012441, 'bagging_freq': 9}


In [45]:
study4 = optuna.create_study(direction='maximize')
study4.optimize(fit_lgbm(x_train[feature_cols4], y_train['target4'], x_valid[feature_cols4], y_valid['target4'], learning_rates[3]), n_trials=100)

print('Number of finished trials:', len(study4.trials))
print('Best trial:', study4.best_trial.params)

[I 2021-07-06 14:52:59,405] A new study created in memory with name: no-name-82a1c662-4957-4a33-a818-1128b4909240


[LightGBM] [Warning] feature_fraction is set=0.886446460563833, colsample_bytree=0.7 will be ignored. Current value: feature_fraction=0.886446460563833
[LightGBM] [Warning] bagging_fraction is set=0.5043419220621458, subsample=0.7 will be ignored. Current value: bagging_fraction=0.5043419220621458
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.849032	valid_1's l1: 1.1059
[200]	training's l1: 0.835957	valid_1's l1: 1.10218
[300]	training's l1: 0.82052	valid_1's l1: 1.10026
[400]	training's l1: 0.80316	valid_1's l1: 1.09643
[500]	training's l1: 0.788533	valid_1's l1: 1.09661


[I 2021-07-06 14:53:16,466] Trial 0 finished with value: -1.0924228604307884 and parameters: {'max_depth': 20, 'min_child_weight': 15, 'subsample': 0.7, 'colsample_bytree': 0.7, 'reg_lambda': 0.00676594570592749, 'reg_alpha': 0.0015373279486903496, 'feature_fraction': 0.886446460563833, 'bagging_fraction': 0.5043419220621458, 'bagging_freq': 10}. Best is trial 0 with value: -1.0924228604307884.


Early stopping, best iteration is:
[485]	training's l1: 0.790428	valid_1's l1: 1.09257
[LightGBM] [Warning] feature_fraction is set=0.8536320813841498, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.8536320813841498
[LightGBM] [Warning] bagging_fraction is set=0.9943352583185832, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9943352583185832
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.846412	valid_1's l1: 1.10131
[200]	training's l1: 0.839955	valid_1's l1: 1.09782
[300]	training's l1: 0.828024	valid_1's l1: 1.09984


[I 2021-07-06 14:53:33,733] Trial 1 finished with value: -1.0964127071403282 and parameters: {'max_depth': 9, 'min_child_weight': 9, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 0.021966667893089986, 'reg_alpha': 113.68004809480458, 'feature_fraction': 0.8536320813841498, 'bagging_fraction': 0.9943352583185832, 'bagging_freq': 5}. Best is trial 0 with value: -1.0924228604307884.


...

Early stopping, best iteration is:
[368]	training's l1: 0.823334	valid_1's l1: 1.09463
[LightGBM] [Warning] feature_fraction is set=0.7970581361401953, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.7970581361401953
[LightGBM] [Warning] bagging_fraction is set=0.49901247606974486, subsample=0.9 will be ignored. Current value: bagging_fraction=0.49901247606974486
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.840729	valid_1's l1: 1.1004
[200]	training's l1: 0.816649	valid_1's l1: 1.08886
[300]	training's l1: 0.802327	valid_1's l1: 1.08963


[I 2021-07-06 15:18:32,200] Trial 98 finished with value: -1.0884631853130344 and parameters: {'max_depth': 5, 'min_child_weight': 12, 'subsample': 0.9, 'colsample_bytree': 0.9, 'reg_lambda': 0.0905140722558326, 'reg_alpha': 0.5576910488728655, 'feature_fraction': 0.7970581361401953, 'bagging_fraction': 0.49901247606974486, 'bagging_freq': 5}. Best is trial 62 with value: -1.085352972838052.


Early stopping, best iteration is:
[221]	training's l1: 0.8133	valid_1's l1: 1.08854
[LightGBM] [Warning] feature_fraction is set=0.8277999658642312, colsample_bytree=0.9 will be ignored. Current value: feature_fraction=0.8277999658642312
[LightGBM] [Warning] bagging_fraction is set=0.5010558829145926, subsample=0.8 will be ignored. Current value: bagging_fraction=0.5010558829145926
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.842669	valid_1's l1: 1.10183
[200]	training's l1: 0.828644	valid_1's l1: 1.09603
[300]	training's l1: 0.813757	valid_1's l1: 1.09509


[I 2021-07-06 15:18:43,999] Trial 99 finished with value: -1.0917947657923386 and parameters: {'max_depth': 5, 'min_child_weight': 11, 'subsample': 0.8, 'colsample_bytree': 0.9, 'reg_lambda': 0.024473583982356134, 'reg_alpha': 1.1681122195229507, 'feature_fraction': 0.8277999658642312, 'bagging_fraction': 0.5010558829145926, 'bagging_freq': 5}. Best is trial 62 with value: -1.085352972838052.


Early stopping, best iteration is:
[272]	training's l1: 0.817737	valid_1's l1: 1.09207
Number of finished trials: 100
Best trial: {'max_depth': 6, 'min_child_weight': 14, 'subsample': 0.6, 'colsample_bytree': 0.9, 'reg_lambda': 0.011231825728715695, 'reg_alpha': 0.001230114349908132, 'feature_fraction': 0.9080303457094819, 'bagging_fraction': 0.44692037828438025, 'bagging_freq': 5}


In [46]:
study1.best_trial.params

{'max_depth': 5,
 'min_child_weight': 15,
 'subsample': 0.6,
 'colsample_bytree': 0.8,
 'reg_lambda': 0.4069187916380186,
 'reg_alpha': 0.3230034165640419,
 'feature_fraction': 0.930062038668566,
 'bagging_fraction': 0.46153099230093236,
 'bagging_freq': 3}

In [47]:
study2.best_trial.params

{'max_depth': 5,
 'min_child_weight': 18,
 'subsample': 0.5,
 'colsample_bytree': 0.6,
 'reg_lambda': 0.003047336161981325,
 'reg_alpha': 0.06817362969639486,
 'feature_fraction': 0.9298564165879744,
 'bagging_fraction': 0.9439577588582982,
 'bagging_freq': 9}

In [48]:
study3.best_trial.params

{'max_depth': 5,
 'min_child_weight': 8,
 'subsample': 0.9,
 'colsample_bytree': 0.5,
 'reg_lambda': 0.018839288270763987,
 'reg_alpha': 1.760156672313524,
 'feature_fraction': 0.4601637468609972,
 'bagging_fraction': 0.6761793533012441,
 'bagging_freq': 9}

In [49]:
study4.best_trial.params

{'max_depth': 6,
 'min_child_weight': 14,
 'subsample': 0.6,
 'colsample_bytree': 0.9,
 'reg_lambda': 0.011231825728715695,
 'reg_alpha': 0.001230114349908132,
 'feature_fraction': 0.9080303457094819,
 'bagging_fraction': 0.44692037828438025,
 'bagging_freq': 5}